In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [3]:
keys = ['iter', 'mAp',
        'R @ 20', 'R @ 50', 'R @ 100',
        'ngR @ 20', 'ngR @ 50', 'ngR @ 100',
        'zR @ 20', 'zR @ 50', 'zR @ 100',
        'mR @ 20', 'mR @ 50', 'mR @ 100',
        'A @ 20', 'A @ 50', 'A @ 100',
       ]
patern_str_measure = 'mAp=([\d\.]+)(?:.|\n)+?R @ 20: ([\d\.]+);   R @ 50: ([\d\.]+);   R @ 100: ([\d\.]+)(?:.|\n)+?.+?@ 20: ([\d\.]+); .+?@ 50: ([\d\.]+); .+?@ 100: ([\d\.]+)(?:.|\n)+?.+?@ 20: ([\d\.]+); .+?@ 50: ([\d\.]+); .+?@ 100: ([\d\.]+)(?:.|\n)+?.+?@ 20: ([\d\.]+); .+?@ 50: ([\d\.]+); .+?@ 100: ([\d\.]+)(?:.|\n)+?.+?@ 20: ([\d\.]+); .+?@ 50: ([\d\.]+); .+?@ 100: ([\d\.]+)'
patern_str_validation = 'iter: (\d+)(?:(?!iter).)*'+patern_str_measure
patern_str_test =   'iteration (\d+)(?:(?!iter).)*'+patern_str_measure

# patern_compiled_patern_str_validation = re.compile(patern_str_validation, re.MULTILINE | re.DOTALL)

In [4]:

import os
import os.path

log_files = {}
for dirpath, dirnames, filenames in os.walk("./checkpoint/"):
    for filename in [f for f in filenames if f=="log.txt"]:
        #print(os.path.join(dirpath, filename))
        #print(dirpath, filename)
        log_files[dirpath[51:]] =  dirpath+'/'+filename
log_files

{'': './checkpoint/motif-precls-exmp/log.txt',
 'dist1-TDE': './checkpoint/precls-CausalAnalysisPredictor-motifs-dist1-TDE/log.txt',
 'dist2-TDE': './checkpoint/precls-CausalAnalysisPredictor-motifs-dist2-TDE/log.txt',
 'gate-TDE': './checkpoint/precls-CausalAnalysisPredictor-motifs-gate-TDE/log.txt',
 'mfb1-TDE': './checkpoint/precls-CausalAnalysisPredictor-motifs-mfb1-TDE/log.txt',
 'mfb2-TDE': './checkpoint/precls-CausalAnalysisPredictor-motifs-mfb2-TDE/log.txt',
 'mfb3-TDE': './checkpoint/precls-CausalAnalysisPredictor-motifs-mfb3-TDE/log.txt',
 'mfb4-TDE': './checkpoint/precls-CausalAnalysisPredictor-motifs-mfb4-TDE/log.txt',
 'mfb5-TDE': './checkpoint/precls-CausalAnalysisPredictor-motifs-mfb5-TDE/log.txt',
 'mfb7-TDE': './checkpoint/precls-CausalAnalysisPredictor-motifs-mfb7-TDE/log.txt',
 'sum-TDE': './checkpoint/precls-CausalAnalysisPredictor-motifs-sum-TDE/log.txt'}

In [5]:
def matches_to_array(matches, _default_dict = {"label": None, "eval_set":"validation"}):
    arr = []
    for match in matches:
        _dict = _default_dict
        for key,val in zip(keys,match):
            _dict[key] = val
        arr.append(_dict. copy())
        #print(_dict)
    return arr


def log_to_array(Tx: str, label=None):
    matches = re.findall(patern_str_validation, Tx,  re.MULTILINE | re.DOTALL)
    arr = matches_to_array(matches, _default_dict = {"label": label, "eval_set":"validation"})
    
    matches = re.findall(patern_str_test, Tx,  re.MULTILINE | re.DOTALL)
    arr2 = matches_to_array(matches, _default_dict = {"label": label, "eval_set":"test"})

    return arr + arr2

In [6]:
arr = []
for label,log_file in log_files.items():
    #log_files = r'C:\Inn\Github\SGG\checkpoint\precls-CausalAnalysisPredictor-motifs-dist2-TDE\log.txt'
    f = open(log_file,mode='r') 
    Tx = f.read()
    f.close()

    arr += log_to_array(Tx, label=label)

df = pd.DataFrame(data=arr)#, columns=keys
df

,label,eval_set,iter,mAp,R @ 20,R @ 50,R @ 100,ngR @ 20,ngR @ 50,ngR @ 100,zR @ 20,zR @ 50,zR @ 100,mR @ 20,mR @ 50,mR @ 100,A @ 20,A @ 50,A @ 100
0,,validation,2000,0.9999,0.6015,0.6554,0.6722,0.6856,0.8146,0.8774,0.0133,0.0178,0.0356,0.1162,0.1411,0.1516,0.6970,0.7018,0.7018
1,,validation,4000,0.9999,0.6032,0.6590,0.6737,0.6913,0.8168,0.8828,0.0133,0.0289,0.0467,0.1218,0.1520,0.1610,0.6985,0.7033,0.7033
2,,validation,6000,0.9999,0.6023,0.6577,0.6736,0.6929,0.8186,0.8849,0.0089,0.0222,0.0267,0.1286,0.1578,0.1680,0.7001,0.7047,0.7047
3,,validation,8000,0.9999,0.6083,0.6601,0.6759,0.6957,0.8224,0.8873,0.0133,0.0267,0.0400,0.1140,0.1395,0.1489,0.7014,0.7062,0.7062
4,,validation,9600,0.9999,0.6082,0.6603,0.6760,0.6954,0.8225,0.8873,0.0133,0.0267,0.0400,0.1140,0.1396,0.1486,0.7016,0.7063,0.7063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,sum-TDE,validation,16000,0.9999,0.2278,0.3229,0.3872,0.2807,0.4605,0.6130,0.0941,0.1593,0.1807,0.1924,0.2588,0.2962,0.4654,0.4698,0.4698
73,sum-TDE,validation,18000,0.9999,0.2346,0.3335,0.3897,0.2832,0.4650,0.6124,0.1030,0.1585,0.1896,0.2040,0.2676,0.3052,0.4567,0.4613,0.4613
74,sum-TDE,validation,20000,0.9999,0.2133,0.3144,0.3778,0.2614,0.4384,0.5939,0.1178,0.1756,0.1978,0.1913,0.2603,0.3006,0.4512,0.4549,0.4549
75,sum-TDE,validation,22000,0.9999,0.2198,0.3194,0.3815,0.2645,0.4354,0.5853,0.1222,0.1637,0.1874,0.1922,0.2617,0.3048,0.4544,0.4585,0.4585


In [7]:
df.columns

Index(['label', 'eval_set', 'iter', 'mAp', 'R @ 20', 'R @ 50', 'R @ 100',
       'ngR @ 20', 'ngR @ 50', 'ngR @ 100', 'zR @ 20', 'zR @ 50', 'zR @ 100',
       'mR @ 20', 'mR @ 50', 'mR @ 100', 'A @ 20', 'A @ 50', 'A @ 100'],
      dtype='object')

In [8]:
df[df["label"]=="mfb7-TDE"]

,label,eval_set,iter,mAp,R @ 20,R @ 50,R @ 100,ngR @ 20,ngR @ 50,ngR @ 100,zR @ 20,zR @ 50,zR @ 100,mR @ 20,mR @ 50,mR @ 100,A @ 20,A @ 50,A @ 100
42,mfb7-TDE,validation,2000,0.9999,0.2988,0.3581,0.3775,0.3911,0.5480,0.6469,0.0911,0.1474,0.1785,0.1478,0.1747,0.1821,0.3755,0.3778,0.3778
43,mfb7-TDE,validation,4000,0.9999,0.2786,0.3434,0.3668,0.3650,0.5460,0.6643,0.0978,0.1785,0.2119,0.1609,0.1889,0.1979,0.3645,0.3672,0.3672
44,mfb7-TDE,validation,6000,0.9999,0.2802,0.3460,0.3675,0.3710,0.5495,0.6648,0.1104,0.1637,0.1963,0.1467,0.1703,0.1819,0.3618,0.3653,0.3653
45,mfb7-TDE,validation,8000,0.9999,0.3029,0.3680,0.3892,0.3976,0.5803,0.6954,0.1126,0.1652,0.1830,0.1650,0.1916,0.2038,0.3915,0.3958,0.3958
46,mfb7-TDE,validation,10000,0.9999,0.3201,0.4006,0.4213,0.4415,0.6873,0.7998,0.1119,0.1600,0.1793,0.1615,0.1887,0.2002,0.4391,0.4437,0.4437
47,mfb7-TDE,validation,12000,0.9999,0.3424,0.4269,0.4441,0.4851,0.7329,0.8111,0.1089,0.1652,0.1919,0.1553,0.1840,0.1934,0.4747,0.4796,0.4796
48,mfb7-TDE,validation,14000,0.9999,0.3843,0.4690,0.4839,0.5332,0.7493,0.8043,0.1104,0.1704,0.1985,0.1522,0.1781,0.1896,0.5271,0.5320,0.5320
49,mfb7-TDE,validation,16000,0.9999,0.3767,0.4675,0.4853,0.5044,0.7160,0.7778,0.1222,0.1911,0.2185,0.1571,0.1868,0.1999,0.5241,0.5288,0.5288
50,mfb7-TDE,validation,18000,0.9999,0.0077,0.0135,0.0182,0.0065,0.0284,0.0642,0.0311,0.0422,0.0556,0.0059,0.0100,0.0138,0.0242,0.0242,0.0242
51,mfb7-TDE,validation,20000,0.9999,0.0753,0.1061,0.1214,0.0830,0.1107,0.1549,0.0778,0.1044,0.1356,0.0503,0.0862,0.1070,0.1231,0.1238,0.1238


In [9]:
import matplotlib.pyplot as plt
plt.figure();
df[df["label"]=="mfb7-TDE"].plot( x='iter', y=['mAp', 'R @ 20', 'R @ 50', 'R @ 100',
       'ngR @ 20', 'ngR @ 50', 'ngR @ 100', 'zR @ 20', 'zR @ 50', 'zR @ 100',
       'mR @ 20', 'mR @ 50', 'mR @ 100', 'A @ 20', 'A @ 50', 'A @ 100'])

TypeError: no numeric data to plot

In [10]:
df[df["label"]=="mfb7-TDE"].iplot(x="iter")

In [11]:
df[df["eval_set"]=="test"]

,label,eval_set,iter,mAp,R @ 20,R @ 50,R @ 100,ngR @ 20,ngR @ 50,ngR @ 100,zR @ 20,zR @ 50,zR @ 100,mR @ 20,mR @ 50,mR @ 100,A @ 20,A @ 50,A @ 100
22,dist2-TDE,test,24000,0.9995,0.5354,0.6005,0.6206,0.6042,0.7359,0.8031,0.0624,0.1092,0.1362,0.1093,0.1429,0.1612,0.6391,0.6412,0.6412
33,gate-TDE,test,20000,0.9995,0.3310,0.4448,0.4940,0.3699,0.5500,0.6699,0.0546,0.0915,0.1196,0.1870,0.2586,0.2968,0.5303,0.5322,0.5322
62,mfb7-TDE,test,40000,0.9995,0.4856,0.5935,0.6386,0.5314,0.6987,0.7996,0.0010,0.0032,0.0044,0.0833,0.1281,0.1555,0.6775,0.6801,0.6801
76,sum-TDE,test,22000,0.9995,0.2503,0.3583,0.4147,0.2947,0.4774,0.6148,0.0765,0.1217,0.1507,0.1735,0.2466,0.2881,0.4617,0.4634,0.4634


In [22]:
_df = df[ df['eval_set'] == "test" ]
#_df= _df.set_index('label')
_df=_df.T.reset_index()
_df.columns = _df.iloc[0]
_df = _df.drop([0,1,2,3]) # drops rows label, eval_set, iter, mAP
_df
#list(_df.iloc[0])

,label,dist2-TDE,gate-TDE,mfb7-TDE,sum-TDE
4,R @ 20,0.5354,0.3310,0.4856,0.2503
5,R @ 50,0.6005,0.4448,0.5935,0.3583
6,R @ 100,0.6206,0.4940,0.6386,0.4147
7,ngR @ 20,0.6042,0.3699,0.5314,0.2947
8,ngR @ 50,0.7359,0.5500,0.6987,0.4774
9,ngR @ 100,0.8031,0.6699,0.7996,0.6148
10,zR @ 20,0.0624,0.0546,0.0010,0.0765
11,zR @ 50,0.1092,0.0915,0.0032,0.1217
12,zR @ 100,0.1362,0.1196,0.0044,0.1507
13,mR @ 20,0.1093,0.1870,0.0833,0.1735


In [23]:
_df.columns = ["label","DIST","GATE","MFB","SUM"]

In [24]:
_df2 = _df[["label","SUM","GATE","MFB","DIST"]]
_df2

,label,SUM,GATE,MFB,DIST
4,R @ 20,0.2503,0.3310,0.4856,0.5354
5,R @ 50,0.3583,0.4448,0.5935,0.6005
6,R @ 100,0.4147,0.4940,0.6386,0.6206
7,ngR @ 20,0.2947,0.3699,0.5314,0.6042
8,ngR @ 50,0.4774,0.5500,0.6987,0.7359
9,ngR @ 100,0.6148,0.6699,0.7996,0.8031
10,zR @ 20,0.0765,0.0546,0.0010,0.0624
11,zR @ 50,0.1217,0.0915,0.0032,0.1092
12,zR @ 100,0.1507,0.1196,0.0044,0.1362
13,mR @ 20,0.1735,0.1870,0.0833,0.1093


In [30]:
fig = _df2.iplot(kind="bar", x=keys, categories='label')
fig.update_layout(yaxis=dict(range=[0,1]))

AttributeError: 'NoneType' object has no attribute 'update_layout'

In [14]:
df.iplot(kind="scatter", theme="white",x="iter",title="precls-CausalAnalysisPredictor-motifs-dist2-TDE")